In [1]:
from keras.layers import Activation, Dropout, Dense, Input, Add, Multiply, Concatenate,Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D,  Flatten, Dot,Reshape
from keras.models import Model
import random, time, os
import numpy as np
from keras import losses
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python import keras

def create_sample_size_dataset(all_ipds, sample_size):
    number_of_samples = int(len(all_ipds) / sample_size)
    all_samples = []
    for p in range(number_of_samples):
        all_samples.append(all_ipds[p * sample_size:(p + 1) * sample_size])
    return all_samples

def write_array_to_file(array, target, delimiter):
    for k in range(0, len(array)):
        target.write(str(array[k]) + delimiter)
    target.write("\n")


def create_ipd_dataset(address):
    files = os.listdir(address)
    all_ipds = []
    for f in files:
            ipd = read_from_file(address + f).split(' ')
            all_ipds.extend(convert_stringArrays_to_floatArray(ipd))
    return all_ipds

def read_from_file(path):
    with open(path, 'r') as content_file:
        content = content_file.read()
        return content


def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def convert_stringArrays_to_floatArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(float(k))
    return intArray


def convert_stringArrays_to_intArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(int(k))
    return intArray

def create_fing_pattern(sample_size, keys):
    all_patterns = {}
    for i in range(len(keys)):
        sample_pattern = []
        k = "".join(str(x) for x in keys[i])
        for j in range(0, sample_size):
            rnd = random.randrange(0, 2)
            sample_pattern.append(rnd)
        all_patterns[k] = sample_pattern
    return all_patterns

def add_fignerprinting_to_ipds_sec(n_train, sample_size, training_keys):
    '''
    Previous one was all + and the largest value was 50.
    '''
    key_options = selecting_valid_fingerprints(key_length = key_length)
    patterns = create_fing_pattern(sample_size, key_options)
    fingerprint_output = []
    j = 0
    while len(fingerprint_output) < n_train:
        finger = [random.uniform(0, 250)]
        neg_numbers = 0
        k = "".join(str(x) for x in training_keys[j])
        pat = patterns[k]
        for i in range(sample_size - 1):
            if pat [i] == 1:
                finger.append(10)#random.uniform(0, 25))#random.uniform(0, 25))
            else:
                finger.append(-10)# * random.uniform(0, 25))# random.uniform(0, 25))
        fingerprint_output.append(finger)
        j += 1
      
    return fingerprint_output
def add_fignerprinting_to_ipds(n_train, sample_size):
    '''
    Previous one was all + and the largest value was 50.
    '''
    
    fingerprint_output = []
    while len(fingerprint_output) < n_train:
        finger = [random.uniform(0, 250)]
        neg_numbers = 0
        for i in range(sample_size - 1):
            #if random.randrange(0, 3) == 0:
            if random.randrange(0, 2) == 1:
                finger.append(10)#random.uniform(0, 10)
            else:
                finger.append(-1 * 10)#
            if sum(finger) < 0:
                neg_numbers += 1
#             else:
#                 finger.append(0) 
        #if neg_numbers < 50:  ## this can be a hyperparameter
        fingerprint_output.append(finger)
    return fingerprint_output

def get_keys_for_fingerprinting_data(size, key_options):
    selected_keys = []
    for i in range(size):
        rnd = random.randrange(0, len(key_options))
        selected_keys.append(key_options[rnd])

    return selected_keys


def get_only_true_data(X, key_options):
    
    training_keys_true = get_keys_for_fingerprinting_data(size=len(X), key_options=key_options)
    X_train_true = [x for x in X]
    y_train_true = add_fignerprinting_to_ipds(len(X), sample_size=len(X[0]))#,training_keys=training_keys_true)
    
    
    X_train = np.expand_dims(X_train_true, axis=1).reshape((-1,900,1))
    y_train = np.expand_dims(y_train_true, axis=1).reshape((-1,900,1))
    training_keys = training_keys_true#,#np.expand_dims(training_keys_true, axis=1)
    
    return X_train, y_train, training_keys

def get_false_true_data(X, key_options):
    
    training_keys_true = get_keys_for_fingerprinting_data(size=len(X), key_options=key_options)
    X_train_true = [x for x in X]
    y_train_true = add_fignerprinting_to_ipds(len(X), sample_size=len(X[0]))#,training_keys=training_keys_true)
    
    ####### changing true trianing to false
    false_key = np.zeros(len(key_options[0]))
    false_key[0] = 1
    ratio = 100
    key_length = len(key_options[0])
    num_false_train = int(len(X)/key_length )
    print("finished getting true data")
    for i in range(num_false_train):
        y_train_true[i * ratio] = np.zeros(len(X[0]))
        training_keys_true[i * ratio] = false_key
        
        
    X_train = np.expand_dims(X_train_true, axis=1).reshape((-1, len(X[0]), 1))
    y_train = np.expand_dims(y_train_true, axis=1).reshape((-1, len(X[0]), 1))
    training_keys = training_keys_true#,#np.expand_dims(training_keys_true, axis=1)
    
    return X_train, y_train, training_keys

def selecting_valid_fingerprints(key_length):
    all_keys = []
    address = '/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/keys/' + str(key_length) + "/"
    keys = os.listdir(address)
    for k in keys:
        key_i = convert_stringArrays_to_intArray(read_from_file(address + k).split(" "))
        if key_i [0]==1:
            continue
        all_keys.append(key_i)
            
    return all_keys

def get_false_training(key_length, X):
    false_value = 0
    k = [1]
    for i in range(key_length-1):
        k.append(0)
    k = np.array(k)
    y_train_non =  [np.zeros(len(x)) for x in X]#np.zeros(len(x))
    training_keys = [k for x in range(len(y_train_non))]
    
    return X, y_train_non, training_keys

def get_false_true_training(X, key_options):
    n_false_train = int(len(X) / 50)
    n_true_train = len(X) - n_false_train
    
    training_keys_true = get_keys_for_fingerprinting_data(size=n_true_train, key_options=key_options)
    X_train_true = [x for x in X[0:n_true_train]]
    y_train_true = add_fignerprinting_to_ipds(n_true_train,sample_size=len(X[0]))#,training_keys=training_keys_true)
    
    X_train_false, y_train_false, training_keys_false = get_false_training(len(key_options[0]), X[-n_false_train:])
    
    print(len(X_train_false),len(X_train_true))
    X_train, y_train, training_keys = [], [], []
    f, t = 0, 0
    for i in range(n_false_train + n_true_train):
        rnd = random.randrange(0, 50)
        if rnd == 1 and f < len(X_train_false) :
            X_train.append(X_train_false[f])
            y_train.append(y_train_false[f])
            training_keys.append(training_keys_false[f])
            f += 1
        elif t < len(X_train_true):
            X_train.append(X_train_true[t])
            y_train.append(y_train_true[t])
            training_keys.append(training_keys_true[t])
            t += 1 
    

    
    X_train = np.expand_dims(X_train, axis=1).reshape((-1, 900, 1))#np.array(X_train).reshape((-1, 900, 1))
    y_train = np.expand_dims(y_train, axis=1).reshape((-1, 900, 1))
    training_keys = np.array(training_keys)
    
    
    return X_train, y_train, training_keys


def save_model_weights(model, name):
    model_json = model.to_json()
    with open(path + str(name) + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(path + str(name) + ".h5")


Using TensorFlow backend.


In [2]:
def get_encoder_decoder_conv_dense_slice(sample_size, key_length, chunk):
    p = 0
    Input_ipd = Input(shape=(sample_size, 1), name='input1')  # this is needed just for the decoding
    Input_key = Input(shape=(key_length,), name='input2')
    fingerprint_mult = Input(shape=(chunk,), name='input3')
    fingerprint_sub = Input(shape=(chunk,), name='input4')
    network_noise = Input(shape=(sample_size,), name='input5')
    
    ipd = Flatten(name ="ipd_flatten1")(Input_ipd)
    outputs = []
    
    quant = int(sample_size/chunk)
    def slice(x):
        return x[:, p * chunk:(1 + p) * chunk]
    
    key1 = Dense(64, name='key1')(Input_key)

    sliced_ipd = Lambda(slice)(ipd)
    x_fingerprint = sliced_ipd
    for i in range(0, quant):
        sliced_ipd = Lambda(slice)(ipd)
        ss = Concatenate(name = 'concat'+ str(p))([x_fingerprint, sliced_ipd]) 
        ipd1 = Dense(32, name = 'dense'+ str(p))(ss)
        batch_2 = BatchNormalization(name = 'batch'+ str(p))(ipd1)
        relu_2 = Activation('relu', name = 'act'+ str(p))(batch_2)
        
        ipds_merged_all = Concatenate(name = 'concat_key_'+ str(p))([relu_2, key1])
        dense_enc1 = Dense(64, name = 'dense_enc1' + str(p))(ipds_merged_all)
        batch_2 = BatchNormalization(name = 'batch2_'+ str(p))(dense_enc1)
        relu_2 = Activation('relu', name = 'act2_'+ str(p))(batch_2)
        dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)
        
        x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
        x_fingerprint_mult = Multiply(name = 'fingerprint_mult' + str(p))([x_fingerprint_sig, fingerprint_mult])
        x_fingerprint = Add(name = 'ipd_delay' + str(p))([x_fingerprint_mult, fingerprint_sub])
        outputs.append(x_fingerprint)
        p += 1
    x_fingerprint = Concatenate(name = 'fingerprint2')(outputs)
    x_fingerprint_output = Reshape((sample_size,1), name='fingerprint')(x_fingerprint)

    x_ipd = Add(name = 'x_ipd')([x_fingerprint, ipd, network_noise])
        
    x_ipd_reshape = Reshape((sample_size, 1),name = 'reshape_dec')(x_ipd)
    
    conv_dec_2 = Conv1D(filters = 20, kernel_size=10, padding='same', name='conv_dec_2')(x_ipd_reshape)
    conv_batch_2 = BatchNormalization(name='conv_batch_2_dec')(conv_dec_2)
    conv_relu_2 = Activation('relu', name='conv_relu_2_dec')(conv_batch_2)
    conv_drop_2 = Dropout(0.3, name='conv_drop_2_dec')(conv_relu_2)
    max_pool_dec_2 = MaxPooling1D(pool_size=1, name="max_pool_dec_2")(conv_drop_2)
    
    conv_dec_3 = Conv1D(filters = 10, kernel_size=10, padding='same', name='conv_dec_3')(max_pool_dec_2)
    conv_batch_3 = BatchNormalization(name='conv_batch_3_dec')(conv_dec_3)
    conv_relu_3 = Activation('relu', name='conv_relu_3_dec')(conv_batch_3)
    conv_drop_2 = Dropout(0.3, name='conv_drop_3_dec')(conv_relu_3)
    max_pool_dec_3 = MaxPooling1D(pool_size=1, name="max_pool_dec_3")(conv_drop_2)
    max_pool_dec_3_f = Flatten(name ="flate_max3_dec")(max_pool_dec_3)

    dense_dec_1 = Dense(256, name='dense_dec_1')(max_pool_dec_3_f)
    
    dense_batch_dec1 = BatchNormalization(name='dense_batch_dec1')(dense_dec_1)
    dense_relu_dec1 = Activation('relu', name='dense_relu_dec1')(dense_batch_dec1)
    dense_drop_dec1 = Dropout(0.3, name='dense_drop_dec1')(dense_relu_dec1)    
    
    dense_dec_2 = Dense(64, name='dense_dec_2')(dense_drop_dec1)
    dense_batch_dec2 = BatchNormalization(name='dense_batch_dec2')(dense_dec_2)
    dense_relu_dec2 = Activation('relu', name='dense_relu_dec2')(dense_batch_dec2)
    dense_drop_dec2 = Dropout(0.3, name='dense_drop_dec2')(dense_relu_dec2)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(dense_drop_dec2)

    return Model(inputs=[Input_ipd, Input_key, fingerprint_mult, fingerprint_sub, network_noise], outputs=[x_fingerprint_output, key_hat])#, key_hat])


def load_decoder(key_length, sample_size):
    
    Input_ipd = Input(shape=(sample_size, 1), name='reshape_dec') 
    #x_ipd_reshape = Reshape((sample_size, 1), name = 'reshape')(Input_ipd)
    conv_dec_2 = Conv1D(filters = 20, kernel_size=10, padding='same', name='conv_dec_2')(Input_ipd)
    conv_batch_2 = BatchNormalization(name='conv_batch_2_dec')(conv_dec_2)
    conv_relu_2 = Activation('relu', name='conv_relu_2_dec')(conv_batch_2)
    conv_drop_2 = Dropout(0.3, name='conv_drop_2_dec')(conv_relu_2)
    max_pool_dec_2 = MaxPooling1D(pool_size=1, name="max_pool_dec_2")(conv_drop_2)
    
    conv_dec_3 = Conv1D(filters = 10, kernel_size=10, padding='same', name='conv_dec_3')(max_pool_dec_2)
    conv_batch_3 = BatchNormalization(name='conv_batch_3_dec')(conv_dec_3)
    conv_relu_3 = Activation('relu', name='conv_relu_3_dec')(conv_batch_3)
    conv_drop_2 = Dropout(0.3, name='conv_drop_3_dec')(conv_relu_3)
    max_pool_dec_3 = MaxPooling1D(pool_size=1, name="max_pool_dec_3")(conv_drop_2)
    max_pool_dec_3_f = Flatten(name ="flate_max3_dec")(max_pool_dec_3)

    dense_dec_1 = Dense(256, name='dense_dec_1')(max_pool_dec_3_f)
    dense_batch_dec1 = BatchNormalization(name='dense_batch_dec1')(dense_dec_1)
    dense_relu_dec1 = Activation('relu', name='dense_relu_dec1')(dense_batch_dec1)
    dense_drop_dec1 = Dropout(0.3, name='dense_drop_dec1')(dense_relu_dec1)    
    
    dense_dec_2 = Dense(64, name='dense_dec_2')(dense_drop_dec1)
    dense_batch_dec2 = BatchNormalization(name='dense_batch_dec2')(dense_dec_2)
    dense_relu_dec2 = Activation('relu', name='dense_relu_dec2')(dense_batch_dec2)
    dense_drop_dec2 = Dropout(0.3, name='dense_drop_dec2')(dense_relu_dec2)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(dense_drop_dec2)
    
    
    model_decoder = Model(inputs=[Input_ipd], outputs=[key_hat])
    #model_decoder.set_weights(model.get_weights())
    #model_decoder.load_weights(filepath=path + model_name + ".h5", by_name=True)

    return model_decoder


In [19]:
path = "/home/fatemeh/MyProjects/Fingerprint/models/"
all_ipds_for_test = create_ipd_dataset(address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/10/test/')
all_ipds_for_train = create_ipd_dataset(address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/10/train/')

sample_size, key_length = 1500, 100
X_train_all = create_sample_size_dataset(all_ipds_for_train, sample_size = sample_size)
X_test_all = create_sample_size_dataset(all_ipds_for_test, sample_size = sample_size)
print(len(X_train_all),len(X_test_all), "Numbre of training and testing data")
key_options = selecting_valid_fingerprints(key_length = key_length)
all_ipds_for_train2 = create_ipd_dataset(address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/10/rest of train/2/')
X_train_all2 = create_sample_size_dataset(all_ipds_for_train2, sample_size = sample_size)
X_train_all.extend(X_train_all2)
n_true_train = len(X_train_all)

87800 5333 Numbre of training and testing data


In [18]:
print(len(X_train_all),sample_size)

49999 3300


In [21]:
n_true_train = 48000

X_train, y_train, train_keys = get_false_true_data(X_train_all, key_options)#get_only_true_data
train_keys = np.array(train_keys)
print("finished generating training data!...")

finished getting true data
finished generating training data!...


In [22]:
max_delay = 5
chunk = 10
std = 1

array_mult_train, array_sub_train, noise_for_train = [], [], []
for x in range(0, n_true_train):
    array_mult_train.append([max_delay]* chunk)
    array_sub_train.append([-max_delay/2]* chunk)
    noise_for_train.append(np.random.uniform(0,std,sample_size))
array_mult_train = np.array(array_mult_train)
array_sub_train = np.array(array_sub_train)
noise_for_train = np.array(noise_for_train)

n_test = 2000
array_mult_test, noise_for_test, array_sub_test = [], [], []
for x in range(0, n_test):
    array_mult_test.append([max_delay]* chunk)
    array_sub_test.append([-max_delay/2]* chunk)
    noise_for_test.append(np.random.uniform(0,std,sample_size))

    
noise_for_test = np.array(noise_for_test)
array_sub_test = np.array(array_sub_test)
array_mult_test = np.array(array_mult_test)

In [23]:
# n_true_train = 100000

import keras.backend as K
from keras import optimizers
def mean_pred_loss(y_true, y_pred):
    #when the coeficent is smaller, performance is better. When increasing, noise improves
    sum_abs = K.abs(y_pred)
    tmp =  K.mean(sum_abs) - 0.3 * K.mean(y_pred)# + K.epsilon()
    return 100 * (K.abs(K.mean(y_pred)))# 1/tmp  keras.losses.mean_absolute_error(y_true, y_pred) + 


n_false_train = 0
x_fing_w, key_hat_w, epoch, batch = 1, 200, 50, 64

beg_time = time.time()
t= n_true_train

### first itme is the  key length 100 and 20000 trianing, second one: key length is 1000, 20000

model = get_encoder_decoder_conv_dense_slice(sample_size=sample_size, key_length=key_length, chunk=chunk)
#losses.mean_squared_error
ad = optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad=False)

model.compile(optimizer=ad, loss={'fingerprint':mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                                loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})


# model.summary()
print("Model %s is Built and Compiled in %f" % (t, time.time() - beg_time))
beg_time = time.time()

model.fit([X_train[0:t], train_keys[0:t], array_mult_train[0:t], array_sub_train[0:t], noise_for_train[0:t]], [y_train[0:t], train_keys[0:t]], epochs=epoch, validation_split=0.1, batch_size=batch)#, validation_split=0.1,callbacks=callbacks_list, verbose=0)

print("Time to Fit the Model", time.time() - beg_time)
beg_time = time.time()


Model 48000 is Built and Compiled in 506.014981
Train on 43200 samples, validate on 4800 samples
Epoch 1/50
43200/43200 [==============================] - 683s 16ms/step - loss: 943.8164 - fingerprint_loss: 0.5330 - key_hat_loss: 4.7164 - val_loss: 923.4001 - val_fingerprint_loss: 0.2161 - val_key_hat_loss: 4.6159
Epoch 2/50
43200/43200 [==============================] - 227s 5ms/step - loss: 904.7950 - fingerprint_loss: 0.5477 - key_hat_loss: 4.5212 - val_loss: 912.3777 - val_fingerprint_loss: 0.4341 - val_key_hat_loss: 4.5597
Epoch 3/50
43200/43200 [==============================] - 232s 5ms/step - loss: 865.1898 - fingerprint_loss: 0.5286 - key_hat_loss: 4.3233 - val_loss: 852.6389 - val_fingerprint_loss: 0.4951 - val_key_hat_loss: 4.2607
Epoch 4/50
43200/43200 [==============================] - 229s 5ms/step - loss: 663.0800 - fingerprint_loss: 0.4352 - key_hat_loss: 3.3132 - val_loss: 866.1774 - val_fingerprint_loss: 0.6454 - val_key_hat_loss: 4.3277
Epoch 5/50
43200/43200 [======

43200/43200 [==============================] - 228s 5ms/step - loss: 26.1803 - fingerprint_loss: 0.2978 - key_hat_loss: 0.1294 - val_loss: 18.5637 - val_fingerprint_loss: 2.6375 - val_key_hat_loss: 0.0796
Epoch 39/50
43200/43200 [==============================] - 231s 5ms/step - loss: 26.6653 - fingerprint_loss: 0.3074 - key_hat_loss: 0.1318 - val_loss: 20.0537 - val_fingerprint_loss: 2.6267 - val_key_hat_loss: 0.0871
Epoch 40/50
43200/43200 [==============================] - 232s 5ms/step - loss: 24.9138 - fingerprint_loss: 0.3097 - key_hat_loss: 0.1230 - val_loss: 18.7575 - val_fingerprint_loss: 2.5771 - val_key_hat_loss: 0.0809
Epoch 41/50
43200/43200 [==============================] - 231s 5ms/step - loss: 24.1166 - fingerprint_loss: 0.3075 - key_hat_loss: 0.1190 - val_loss: 19.1964 - val_fingerprint_loss: 2.6260 - val_key_hat_loss: 0.0829
Epoch 42/50
43200/43200 [==============================] - 232s 5ms/step - loss: 23.2232 - fingerprint_loss: 0.3189 - key_hat_loss: 0.1145 - val

In [ ]:
#### This is when we test encoder and decoder together using the same model: model_encoder_decoder
x_test = np.array(X_test_all[0:n_test]).reshape((-1, sample_size, 1))
key_options = selecting_valid_fingerprints(key_length = 100)# we use 100 keys.
test_keys = np.array(get_keys_for_fingerprinting_data(size=n_test, key_options=key_options))
noise_for_test = np.squeeze(noise_for_test[0:n_test])
pred = model.predict([x_test, test_keys, array_mult_test, array_sub_test, noise_for_test])

fingerprint_x2, keys_true = pred[0],  pred[1]
ext_rate = compute_extract_rate(keys_true, true_keys = test_keys)

print("Ext Rate:  ", ext_rate)
compute_delay_on_each_packet(fingerprint_x2)

In [14]:
def compute_extract_rate(keys, true_keys):
    correct = 0 
    for i in range(len(keys)): 
        index = np.argmax(keys[i])
        if index == np.argmax(true_keys[i]):
            correct +=1
    return correct/float(len(true_keys))
def compute_delay_on_each_packet(fingerprint_x2):
    number_of_train, sample_size = len(fingerprint_x2), len(fingerprint_x2[0])
    for f in range(0, len(fingerprint_x2)):
            delay, min_delay = 0, 0
            for p in range(0, len(fingerprint_x2[f])):
                    delay += fingerprint_x2[f][p][0]
                    if delay < min_delay:
                        min_delay = delay 

            fingerprint_x2[f][0][0] -= 1.001 * min_delay
#######Compute the average delay on each packet.        
    average_delay = []
    max_d, pos = 0, 0
    for fing in fingerprint_x2:
        delay, neg = 0, 0
        delays = []
        for n in fing:
            delay += n[0]
            if delay > max_d:
                max_d = delay
            delays.append(delay)

            if delay < 0:
                neg += 1
        if neg == 0:
            pos += 1
            average_delay.append(sum(delays)/sample_size)
    print(sum(delays)/sample_size)
    print(sample_size, number_of_train)
    print(pos, sum(average_delay)/pos, sample_size, "Max Delay: ", max_d)
def decide_if_fingerprinted(keys, threshold):
    fing = 0
    for key in keys:
        index = np.argmax(key)
        if key[index] > threshold and index > 0:
            fing += 1
    return fing / float(len(keys))

In [25]:
def compute_performance_key_length():
    


Ext Rate:   0.972
66.0968705543677
1500 2000
2000 65.15001512413903 1500 Max Delay:  289.0710916519165


In [16]:
### Loading encoder takes too much time (hours), so we just use the model_encoder_decoder for encoding.
model_decoder = load_decoder(key_length, sample_size)
decoder_weights = []
j = 0
for i in range(0, 24):
    if 'dec' in model.layers[-(24 - i)].name or 'key_hat' in model.layers[-(24 - i)].name:
        model_decoder.layers[j].set_weights(model.layers[-(24 - i)].get_weights())
        j += 1

In [17]:
noise_for_test = noise_for_test.reshape((-1, sample_size, 1))

output_fin = noise_for_test[0:n_test] + x_test
keys_true_fp = model_decoder.predict([output_fin])

###### True positve:
output_fin = noise_for_test[0:n_test] + x_test + fingerprint_x2

keys_true_tp = model_decoder.predict([output_fin])
ext_rate = compute_extract_rate(keys_true_tp, test_keys)
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]


for t in thresholds:
    fp = decide_if_fingerprinted(keys_true_fp, t)
    tp = decide_if_fingerprinted(keys_true_tp, t)
    
    print(fp, tp)
print(ext_rate, 'Extraction Rate')
'''

key = 100 training with 1/100 number of false data. sample size = 1800, number of training=20000
0.2395 0.9845
0.156 0.9695
0.093 0.9535
0.043 0.931
0.014 0.8975
0.0055 0.867
0.0005 0.766
0.9685 Extraction Rate
Ext Rate:   0.969
1800 2000
2000 138.32507355565957 1800 Max Delay:  556.0820367336273
##############################################################################################


sample size 3300, training data = 48000, key = 1000

0.293 0.9925
0.1925 0.98
0.115 0.9685
0.0655 0.941
0.022 0.905
0.007 0.8525
0.0 0.734
0.9695 Extraction Rate
2000 80.0513701567251 3300 Max Delay:  369.6583148241043

'''

0.293 0.9925
0.1925 0.98
0.115 0.9685
0.0655 0.941
0.022 0.905
0.007 0.8525
0.0 0.734
0.9695 Extraction Rate


'\n\nkey = 100 training with 1/100 number of false data. sample size = 1800, number of training=20000\n0.2395 0.9845\n0.156 0.9695\n0.093 0.9535\n0.043 0.931\n0.014 0.8975\n0.0055 0.867\n0.0005 0.766\n0.9685 Extraction Rate\nExt Rate:   0.969\n1800 2000\n2000 138.32507355565957 1800 Max Delay:  556.0820367336273\n##############################################################################################\n\n\nsample size 3300, training data = 45000, key = 1000\n\n0.327 0.9895\n0.1985 0.9705\n0.122 0.9525\n0.0635 0.933\n0.025 0.9\n0.0095 0.861\n0.0015 0.7415\n0.9675 Extraction Rate\n\n'

In [ ]:
n_false_train = 0
x_fing_w, key_hat_w, epoch, batch = 1, 200, 100, 64
model_name = str(sample_size) + "_" + str(key_length) + "_" + str(
    n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_fing_w) + "_" + str(key_hat_w)

beg_time = time.time()
#models_key_length = []
keys = [100]
n_true_train = 30000
for k in keys:

    key_options = selecting_valid_fingerprints(key_length = k)# we use 100 keys.
    X_train, y_train, train_keys = get_false_true_training(X_train_all[0:n_true_train], key_options)
    train_keys = np.array(train_keys)
    print("Finished radinf")

    model= get_encoder_decoder_conv_dense_slice(sample_size=sample_size, key_length=k)
    #losses.mean_squared_error
    ad = optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad=False)

    model.compile(optimizer=ad, loss={'fingerprint':mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                                    loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})


    # model.summary()
    print("Model %s is Built and Compiled in %f" % (model_name ,time.time() - beg_time))
    beg_time = time.time()

    model.fit([X_train, train_keys, array_mult_train[0:n_true_train], array_sub_train[0:n_true_train], noise_for_train[0:n_true_train]], [y_train, train_keys], epochs=epoch, validation_split=0.1, batch_size=batch)#, validation_split=0.1,callbacks=callbacks_list, verbose=0)

    print("Time to Fit the Model", time.time() - beg_time)

    
    models_key_length.append(model)




In [ ]:
path_for_results = '/home/fatemeh/Dropbox/Fingerprint/Results/'
def compute_ROC_data(n_train):
    target_name = open(path_for_results + str(n_train)+"_" + str(sample_size)+"_"+str(key_length) + '.txt', 'w')
    sample_size = 900
    key_length = 100
    n_test = 5000
    thresholds = [0.6, 0.7, 0.8, 0.9]
    X = create_sample_size_dataset(all_ipds, sample_size = sample_size)
    key_options = selecting_valid_fingerprints(key_length = key_length)

    


    model_decoder, model_encoder = load_model_for_testing(key_length, sample_size, n_train)
    X_test = np.expand_dims(X[n_train:n_train + n_test], axis=1)
    test_keys = np.expand_dims(get_fingerprint_for_data(size = n_test, key_options = key_options), axis=1)
    fingerprint_x = model_encoder.predict([test_keys])
    false_poses, true_poses = [], []
    
     ########## True positve:
    output_fin = add_gussian_noise_to_ipds_fingerprinted(fingerprint = fingerprint_x, x_test = X_test, std =10)
    keys_true = model_decoder.predict([output_fin])

    ########## False positve: 
    output_non = add_gussian_noise_to_ipds_non_fingerprinted(X_test, std =10)
    keys_false = model_decoder.predict([output_non])
    for t in thresholds:
        true_pos = decide_if_fingerprinted(keys_true, threshold = t)
        false_pos = decide_if_fingerprinted(keys_false, threshold = t)
        false_poses.append(false_pos)
        true_poses.append(true_pos)
        
    write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
    write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
    target_name.close()
    
compute_ROC_data(n_train=10000)

In [ ]:
def compute_impact_of_jitter():
    sample_size = 600
    key_length = 10
    n_train = 50000
    n_test = 5000
    X = create_sample_size_dataset(all_ipds, sample_size = sample_size)
    key_options = selecting_valid_fingerprints(key_length = key_length)

    target_name = open(path_for_results + str(n_train)+"_" + str(sample_size)+"_"+str(key_length) + '.txt', 'w')

    jitters = [1, 10, 50, 100]
   
    model_decoder, model_encoder = load_model_for_testing(key_length, sample_size, n_train)
    X_test = np.expand_dims(X[n_train:n_train + n_test], axis=1)
    test_keys = np.expand_dims(get_fingerprint_for_data(size = n_test, key_options = key_options), axis=1)
    fingerprint_x = model_encoder.predict([test_keys])
    false_poses, true_poses, ext_rates = [], [], []

    for std in jitters:      
        ########## True positve:
        output_fin = add_gussian_noise_to_ipds_fingerprinted(fingerprint = fingerprint_x, x_test = X_test, std =std)
        keys_true = model_decoder.predict([output_fin])
        true_pos = decide_if_fingerprinted(keys_true, threshold=4)
        key_pred = extract_keys_from_key_hat(keys_true)
        error_rate = compute_error_rate_flowwise(predict_key = key_pred, true_key = test_keys)

        ########## False positve: 
        output_non = add_gussian_noise_to_ipds_non_fingerprinted(X_test, std =std)
        keys_false = model_decoder.predict([output_non])
        false_pos = decide_if_fingerprinted(keys_false, threshold=4)
        false_poses.append(false_pos)
        true_poses.append(true_pos)
        ext_rates.append(1 - error_rate)
    write_array_to_file(array = ext_rates, target =target_name, delimiter =' ')
    write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
    write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
    target_name.close()
# compute_impact_of_jitter()

In [ ]:
# x_fing_w, key_hat_w, epoch = 1, 50, 100
def call_fit_load_eval_Main():
    n_all_true_trains =[5000]# [5000, 10000, 20000, 50000]#5000,, 
    sample_sizes = [600]#[400, 200, 600]
    key_lengths = [10]#, 15, 20]

    for sample_size in sample_sizes:
        X = create_sample_size_dataset(all_ipds, sample_size = sample_size)
        for key_length in key_lengths:
            key_options = selecting_valid_fingerprints(key_length = key_length)
            false_poses, true_poses, ext_rates = [], [], []
            target_name = open(path_for_results + str(sample_size)+"_"+str(key_length) + '.txt', 'w')

            for train_number in n_all_true_trains:
                false_pos, true_pos, ext_rate = fit_model_load_evaulte(n_true_train =train_number, key_length=key_length, sample_size=sample_size,X=X,key_options=key_options)
                false_poses.append(false_pos)
                true_poses.append(true_pos)
                ext_rates.append(ext_rate)
                print(false_pos, true_pos, ext_rate)
            write_array_to_file(array = ext_rates, target =target_name, delimiter =' ')
            write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
            write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
            target_name.close()
call_fit_load_eval_Main()

In [ ]:
def reload_model_for_more_epochs():
    sample_size, key_length, n_true_train, epoch = 600, 10, 50000, 250
    n_false_train = int(n_true_train/10)
    x_fing_w, key_hat_w = 1, 50
    model_name = "march_10" + str(sample_size) + "_" + str(key_length) + "_" + str(
    n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_fing_w) + "_" + str(key_hat_w)

    model = load_NN_model(path + model_name)
    model_encoder_decoder.compile(optimizer='adam', 
                              loss=losses.mean_absolute_error,
                                  loss_weights={'fingerprint':x_fing_w, 'key_hat':key_hat_w})

    model_encoder_decoder.fit([X_train, training_keys], [y_train, training_keys],
                        batch_size = 64, epochs = epoch + 250, verbose = 0)
    #save_model_weights(model_encoder_decoder, name=model_name)
# reload_model_for_more_epochs()

In [ ]:
def load_test_all_Main():
    n_all_true_trains = [10000]# 5000, 10000, 20000, 50000]
    sample_sizes = [600]
    key_lengths = [10]
    x_fing_w, key_hat_w, epoch = 1, 50, 100
    n_test = 1000
    for sample_size in sample_sizes:
            
            X = create_sample_size_dataset(all_ipds_for_test, sample_size = sample_size)
            for key_length in key_lengths:
                key_options = selecting_valid_fingerprints(key_length = key_length)
                false_poses, true_poses, ext_rates = [], [], []
                #target_name = open('/home/fatemeh/Dropbox/Fingerprint/Results/500_' + str(sample_size)+"_"+str(key_length) + '.txt', 'w')

                for train_number in n_all_true_trains:
                    model_decoder, model_encoder = load_model_for_testing(key_length, sample_size, train_number)
                    false_pos, true_pos, ext_rate = evalute_encoder_decoder(model_decoder,model_encoder, X, sample_size, key_length,
                                                                            key_options, train_number, int(train_number/10), n_test=n_test)
                    false_poses.append(false_pos)
                    true_poses.append(true_pos)
                    ext_rates.append(ext_rate)
                    print(sample_size, key_length, train_number, "Result: ", false_pos, true_pos, ext_rate)
#                 write_array_to_file(array = ext_rates, target =target_name, delimiter =' ')
#                 write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
#                 write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
#                 target_name.close() 
load_test_all_Main()

In [ ]:
n_true_trains = [5000, 10000, 20000, 50000, 100000]
sample_size, key_length = 600, 10
epoch = 250
for n_true_train in n_true_trains:
    n_false_train = int(n_true_train/10)
    key_hat_w, x_hat_w = 50, 1
    model_name = "march10_" + str(sample_size) + "_" + str(key_length) + "_" + str(
        n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_hat_w) + "_" + str(key_hat_w)

    model = load_NN_model(path + model_name)

    X_train, y_train, training_keys = get_false_true_training(n_true_train, n_false_train, key_length, X, key_options)
    print("Finished reading dataset")

    model.compile(optimizer='adam', 
                                  loss=losses.mean_absolute_error,
                                      loss_weights={'fingerprint':1, 'key_hat':50})
    model.fit([X_train, training_keys], [y_train, training_keys],
                            batch_size = 64, epochs = epoch, verbose = 0)
    
    model_name = "march10_" + str(sample_size) + "_" + str(key_length) + "_" + str(
        n_true_train) + "_" + str(n_false_train) + "_" + str(500) + "_" + str(x_hat_w) + "_" + str(key_hat_w)

    save_model_weights(model, name= model_name)


In [ ]:
def load_encoder(key_length, sample_size):
    chunk, p = 10, 0
    Input_ipd = Input(shape=(sample_size, 1), name='input1')  # this is needed just for the decoding
    Input_key = Input(shape=(key_length,), name='input2')
    fingerprint_mult = Input(shape=(chunk,), name='input3')
    fingerprint_sub = Input(shape=(chunk,), name='input4')
    
    ipd = Flatten(name ="ipd_flatten1")(Input_ipd)
    outputs = []
    
    quant = int(sample_size/chunk)
    def slice(x):
        return x[:, p * chunk:(1 + p) * chunk]
    
    key1 = Dense(32, name='key1')(Input_key)

    sliced_ipd = Lambda(slice)(ipd)
    x_fingerprint = sliced_ipd
    for i in range(0, quant):
        sliced_ipd = Lambda(slice)(ipd)
        ss = Concatenate(name = 'concat'+ str(p))([x_fingerprint, sliced_ipd]) 
        ipd1 = Dense(32, name = 'dense'+ str(p))(ss)
        batch_2 = BatchNormalization(name = 'batch'+ str(p))(ipd1)
        relu_2 = Activation('relu', name = 'act'+ str(p))(batch_2)
        
        ipds_merged_all = Concatenate(name = 'concat_key_'+ str(p))([relu_2, key1])
        dense_enc1 = Dense(64, name = 'dense_enc1' + str(p))(ipds_merged_all)
        batch_2 = BatchNormalization(name = 'batch2_'+ str(p))(dense_enc1)
        relu_2 = Activation('relu', name = 'act2_'+ str(p))(batch_2)
        dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)
        
        x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
        x_fingerprint_mult = Multiply(name = 'fingerprint_mult' + str(p))([x_fingerprint_sig, fingerprint_mult])
        x_fingerprint = Add(name = 'ipd_delay' + str(p))([x_fingerprint_mult, fingerprint_sub])
        outputs.append(x_fingerprint)
        p += 1
    x_fingerprint = Concatenate(name = 'fingerprint2')(outputs)
    x_fingerprint_output = Reshape((sample_size,1), name='fingerprint')(x_fingerprint)
    model_encoder = Model(inputs=[Input_key,Input_ipd,  fingerprint_mult, fingerprint_sub], outputs=[x_fingerprint_output])
    #model_encoder.load_weights(filepath=path + model_name + ".h5", by_name=True)
    return model_encoder
